In [1]:
import soundfile as sf
import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa

c:\Users\bcheo\anaconda3\envs\htx\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load pretrained model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You s

In [12]:
# librispeech_samples_ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
# load audio

file_path = "common_voice/cv-valid-dev/cv-valid-dev/sample-002782.mp3"
# load audio
audio_input, sample_rate = sf.read(file_path)
audio_resampled = librosa.resample(audio_input, orig_sr=sample_rate, target_sr=16000)

    
# load dummy dataset and read soundfiles
# ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

# tokenize
input_values = processor(audio_resampled, sample_rate=16000, return_tensors="pt", padding="longest").input_values  # Batch size 1

# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

print(transcription)


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


["IT DIDN'T BRING WITH IT THE SMELL OF THE DESERT NOR THE THREAT OF MOORISH INVASION IT DIDN'T BRING WITH IT THE SMELL OF THE DESERT NOR THE THREAT OF MOORISH INVASION"]


In [15]:

# Load the audio file
file_path = "common_voice/cv-valid-dev/cv-valid-dev/sample-002557.mp3"
audio_data, sample_rate = sf.read(file_path)
audio_data = librosa.resample(audio_data, orig_sr=sample_rate, target_sr=16000)
sample_rate = 16000

# Define chunk size (e.g., 10 seconds)
chunk_size_seconds = 10
chunk_size_samples = chunk_size_seconds * sample_rate

# # Initialize model and processor
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

# Process each chunk separately
result = ""
for i in range(0, len(audio_data), chunk_size_samples):
    chunk = audio_data[i:i+chunk_size_samples]

    # Tokenize and transcribe the chunk
    input_values = processor(chunk, return_tensors="pt", sampling_rate=sample_rate).input_values
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    result += transcription[0] + " "

# Combine transcriptions from all chunks
# final_transcription = " ".join(transcriptions)
print(result)
# print(final_transcription)


ALL THAT TIME THE MARTIANS MUST HAVE BEEN GETTING READY     ME  


In [31]:
import pandas as pd

df = pd.read_csv("cv-valid-dev.csv")
df.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration,generated_text
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN,NaN
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they se...,2,0,NaN,NaN,NaN,NaN,NaN
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage ent...,2,0,NaN,NaN,NaN,NaN,NaN
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be des...,3,0,NaN,NaN,NaN,NaN,NaN
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could he...,1,0,fourties,female,england,NaN,NaN


In [10]:
import os

for audio in os.listdir("common_voice/cv-valid-dev/cv-valid-dev"):
    # get audio duration
    info = sf.info(os.path.join("common_voice/cv-valid-dev/cv-valid-dev", audio))
    duration = info.duration
    
    if duration > 20:
        print(audio, duration)
    

sample-001603.mp3 26.308125
sample-002557.mp3 60.796125
sample-002782.mp3 28.468125
sample-003047.mp3 22.948125
sample-003273.mp3 25.348125


In [13]:
df.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration,new_column
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN,None
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they se...,2,0,NaN,NaN,NaN,NaN,None
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage ent...,2,0,NaN,NaN,NaN,NaN,None
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be des...,3,0,NaN,NaN,NaN,NaN,None
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could he...,1,0,fourties,female,england,NaN,None


In [26]:
import pandas as pd

# Sample DataFrame
df = pd.DataFrame({'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})

# Row and column index where you want to add the value
row_index = 1
column_name = 'B'

# Value to add
new_value = 10

# Add the value to the specified location
df.at[row_index, column_name] = new_value

print(df)


   A   B  C
0  1   4  7
1  2  10  8
2  3   6  9
